In [1]:
#import necessary packages
from operator import rshift
import cv2 as cv 
import numpy as np
import mediapipe as mp 
import math
import time
import csv
import pandas as pd
from moviepy.editor import *

In [2]:
# ############################################################################
# Function Name: Euclidean Distance for left, right and center
# Input Parameters: 2 eye points, here we are taking right points
# Return Parameters: Distance
# Action: With the 2 eye points finding the square root of a function with the 
#         predefined formula.    
# ############################################################################


def euclidean_distance(point1, point2):
    x1, y1 = point1.ravel()                                #ravel to change 2d or more into a flattened array
    x2, y2 = point2.ravel()
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)  #distance formula
    return distance

In [3]:
# ############################################################################
# Function Name: landmarksDetection
# Input Parameters: frame, face mesh parameters for eye open and close
# Return Parameters: landmarks on a eye
# Action: Drawing a landmark points on a eye    
# ############################################################################


def landmarksDetection(img, results, draw=False):
    img_height, img_width= img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark]
    if draw :
        [cv.circle(img, p, 2, (0,255,0), -1) for p in mesh_coord]

    # returning the list of tuples for each landmarks 
    return mesh_coord

In [4]:
# ############################################################################
# Function Name: Euclidean Distance for a eye open and close
# Input Parameters: 2 eye points
# Return Parameters: Distance
# Action: With the 2 eye points finding the square root of a function with the 
#         predefined formula.
# ############################################################################


def euclaideanDistance(point, point1):
    x, y = point
    x1, y1 = point1
    distance = math.sqrt((x1 - x)**2 + (y1 - y)**2)
    return distance

In [5]:
# ############################################################################
# Function Name: Eye blink Ratio
# Input Parameters: frame, landmarks for a eye, right eye and left eye indices
# Return Parameters: Ratio for a eye blinking
# Action: With the eye landmarks and the eye indices finding the euclidean
#         distance and applying math rules
# ############################################################################


def blinkRatio(img, landmarks, right_indices, left_indices):
    # Right eyes 
    # horizontal line 
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line 
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]
    # draw lines on right eyes 
    cv.line(img, rh_right, rh_left, (254,250,200), 2)
    cv.line(img, rv_top, rv_bottom, (150,150,150), 2)

    # LEFT_EYE 
    # horizontal line 
    lh_right = landmarks[left_indices[0]]
    lh_left = landmarks[left_indices[8]]

    # vertical line 
    lv_top = landmarks[left_indices[12]]
    lv_bottom = landmarks[left_indices[4]]

    rhDistance = euclaideanDistance(rh_right, rh_left)
    rvDistance = euclaideanDistance(rv_top, rv_bottom)

    lvDistance = euclaideanDistance(lv_top, lv_bottom)
    lhDistance = euclaideanDistance(lh_right, lh_left)

    reRatio = rhDistance/rvDistance
    leRatio = lhDistance/lvDistance

    ratio = (reRatio+leRatio)/2
    return ratio 

In [6]:
# ############################################################################
# Function Name: colorBackgroundText
# Input Parameters: frame, text, font, fontScale, textPos, textThickness,
#                   textColor, bgColor, padding
# Return Parameters: frame with all the input parameters
# Action: For the attractive text design we are changing the size, colour
#         style and padding of the text
# ############################################################################


def colorBackgroundText(img, text, font, fontScale, textPos, textThickness=1,textColor=(0,255,0), bgColor=(0,0,0), pad_x=3, pad_y=3):
    (t_w, t_h), _= cv.getTextSize(text, font, fontScale, textThickness)            # getting the text size
    x, y = textPos
    cv.rectangle(img, (x-pad_x, y+ pad_y), (x+t_w+pad_x, y-t_h-pad_y), bgColor,-1) # draw rectangle 
    cv.putText(img,text, textPos,font, fontScale, textColor,textThickness )        # draw in text

    return img

In [7]:
# ############################################################################
# Function Name: textwithBackground
# Input Parameters: frame, text, font, fontScale, textPos, textThickness, 
#                   textColor, bgColor, padding
# Return Parameters: frame with all the input parameters
# Action: For the attractive text design we are changing the size, colour
#         style and padding of the text
# ############################################################################


def textWithBackground(img, text, font, fontScale, textPos, textThickness=1,textColor=(0,255,0), bgColor=(0,0,0), pad_x=3, pad_y=3, bgOpacity=0.5):
    (t_w, t_h), _= cv.getTextSize(text, font, fontScale, textThickness) # getting the text size
    x, y = textPos
    overlay = img.copy() # coping the image
    cv.rectangle(overlay, (x-pad_x, y+ pad_y), (x+t_w+pad_x, y-t_h-pad_y), bgColor,-1) # draw rectangle 
    new_img = cv.addWeighted(overlay, bgOpacity, img, 1 - bgOpacity, 0) # overlaying the rectangle on the image.
    cv.putText(new_img,text, textPos,font, fontScale, textColor,textThickness ) # draw in text
    img = new_img

    return img

In [8]:
# ############################################################################
# Function Name: textwithBackground
# Input Parameters: frame, text, font, fontScale, textPos, textThickness, 
#                   textColor, bgColor, padding
# Return Parameters: frame with all the input parameters
# Action: For the attractive text design we are changing the size, colour
#         style and padding of the text
# ############################################################################


def iris_position(iris_center, right_point, left_point, top_point, bottom_point): #function definition for iris_position
    center_to_right_dist = euclidean_distance(iris_center, right_point)  #euclidean distance calculations
    total_distance = euclidean_distance(right_point, left_point)
    #center_to_top_dist = euclidean_distance(iris_center, top_point)
    #total_distances = euclidean_distance(top_point, bottom_point)
    #ratioupdown = center_to_top_dist/total_distance
    ratio = center_to_right_dist/total_distance      #here we are finding for one eye, that will take u to the second eye automatically
    iris_position = ""                               #initially empty string for position
    if ratio<= 0.42:                                 #comparing with the landmark chart to get the prediciton
        iris_position = "right"                      #if position less than or equal to 0.42 its right
    elif ratio>0.42 and ratio <= 0.57:
        iris_position = "center"                     #if position lies between 0.42 and 0.57 its center
    #elif ratio > 0.56 and ratio <= 0.60:
        #iris_position = "top"
    else:
        iris_position = "left"                       #if none matches consider as left
    return iris_position, ratio 

      #0.42 right, ratio>0.42 and ratio <= 0.57      #returning position and ratio

In [9]:
# ############################################################################
# Function Name: Iris tracking 
# Input Parameters: filepath and filename
# Return Parameters: list of iris location and eye blink
# Action: It shows the frame of a candidate video with all the inputs and
#         and iris movements 
# ############################################################################


def iris_tracking(filepath, filename):
    iris_eye_loc = []
    iris_loc = []
    frame_counter =0
    CLOSED_EYES_FRAME =3
    FONTS =cv.FONT_HERSHEY_COMPLEX
    mp_face_mesh = mp.solutions.face_mesh #face mesh object
    
    eye_closeopen = []
    present_absent = []
    
    LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ] #left eye indices
    RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  # right eye indices
    RIGHT_IRIS = [474,475, 476, 477]                                                            #left iris indices
    LEFT_IRIS = [469, 470, 471, 472]                                                            #right iris indices
    L_H_LEFT =[33]                                            #right eye right most landmark
    L_H_RIGHT = [133]                                         #right eye left most landmark
    R_H_LEFT = [362]                                          #left eye right most landmark
    R_H_RIGHT = [263]                                         #left eye left most landmark
    TOP_LEFT = [159]                                          #left eye top landmark
    BOTTOM_LEFT = [145]                                       #left eye bottom landmark
    TOP_RIGHT = [386]                                         #right eye top landmark
    BOTTOM_RIGHT = [374]                                      #right eye bottom landmark
        
    cap = cv.VideoCapture(filepath+filename)                  #for getting the saved video file

    #    fpss = cap.get(cv.CAP_PROP_FPS)cap.set(cv.CAP_PROP_FPS, 60)
    fpss = cap.get(cv.CAP_PROP_FPS)
    
    with mp_face_mesh.FaceMesh(
        max_num_faces=1,                                      #number of faces we want to detect
        refine_landmarks=True,                                #landmarks for face
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as face_mesh:
        
        start_time = time.time()
        
        while True:
            ret, frame = cap.read()                           #ret - validity of a frame either it is available or not
                                                              #reading a video file
            if not ret:                                       #if there is a last frame in video we have to quit 
                break
                
            frame = cv.flip(frame, 1)                         #flipping the image - to convert a camera into a mirror image
            rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)  #chaging the color for a image to bgr to rgb
            img_h, img_w = frame.shape[:2]                    #to get the height and width 
            
            results = face_mesh.process(rgb_frame)            #for processing the colour conversion
            if results.multi_face_landmarks:                  #to check if all the landmarks are correctly positioned
                mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
                                                              #to get the pixel coordinates
                (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
                (r_cx, r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])
                center_left = np.array([l_cx, l_cy], dtype=np.int32)
                center_right = np.array([r_cx, r_cy], dtype=np.int32)
                cv.circle(frame, center_left, int(l_radius), (255,0,255), 1, cv.LINE_AA)   #circle to capture left iris
                cv.circle(frame, center_right, int(r_radius), (255,0,255), 1, cv.LINE_AA)  #circle to capture right iris
                
                
                #cv.circle(frame, mesh_points[R_H_RIGHT][0], 3, (255, 255, 255), -1, cv.LINE_AA)
                #cv.circle(frame, mesh_points[R_H_LEFT][0], 3, (0, 255, 255), -1, cv.LINE_AA)
                iris_pos, ratio = iris_position(center_right, mesh_points[R_H_RIGHT], mesh_points[R_H_LEFT][0], mesh_points[TOP_RIGHT], mesh_points[BOTTOM_RIGHT][0])
                iris_loc.append(iris_pos) 
                #print(iris_pos)
                    
                cv.putText(
                    frame, 
                    f"Iris_pos: {iris_pos} {ratio:.2f}", 
                    (30, 90),               
                    cv.FONT_HERSHEY_PLAIN,  #font style
                    1.5, 
                    (234, 0, 0),            #color of a text
                    2,                      #thickness
                    cv.LINE_AA)
                present_absent.append('present')
                #print(present)
            else:
                present_absent.append('absent')
                #print(absent)
            
            frame_counter =0
            CLOSED_EYES_FRAME =3
            FONTS =cv.FONT_HERSHEY_COMPLEX
            
            frame_counter +=1
            
            frame = cv.resize(frame, None, fx=1.5, fy=1.5, interpolation=cv.INTER_CUBIC)
            frame_height, frame_width= frame.shape[:2]
            rgb_frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
            results  = face_mesh.process(rgb_frame)
            if results.multi_face_landmarks:
                mesh_coords = landmarksDetection(frame, results, False)
                ratio = blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)
                # cv.putText(frame, f'ratio {ratio}', (100, 100), FONTS, 1.0, utils.GREEN, 2)
                colorBackgroundText(frame,  f'Ratio : {round(ratio,2)}', FONTS, 0.7, (30,100),2, (147,20,255), (0,255,255))

                eye_openclose = ''
                if ratio > 4.5:
                    eye_openclose = "EyeClosed"
                    eye_closeopen.append(eye_openclose)
                    #print("Eyes Closed")
                    colorBackgroundText(frame,  f'Eyes Closed', cv.FONT_HERSHEY_PLAIN, 1.7, (30, 120), 2, (0,255,255))
                else:
                    eye_openclose = "EyeOpen"
                    eye_closeopen.append(eye_openclose)
                    colorBackgroundText(frame,  f'Eyes Open', cv.FONT_HERSHEY_PLAIN, 1.7, (30, 120), 2, (0,255,255))
                    #print("Eyes Open")


            # calculating  frame per seconds FPS
            end_time = time.time()-start_time
            fps = frame_counter/end_time
            #print(fps)
            
            frame =textWithBackground(frame,f'FPS: {round(fps,1)}',FONTS, 1.0, (30, 50), bgOpacity=0.9, textThickness=2)
            # writing image for thumbnail drawing shape
            #cv.imwrite(f'img/frame_{frame_counter}.png', frame)
            
            #frame = cv.rotate(cap, cv.ROTATE_90_CLOCKWISE)
            #frame = cv.flip(frame, -1)
            
            seconds = end_time - start_time
            #print ("Time taken : {0} seconds".format(seconds))
            #print("Estimated frames per second : {0}".format(fps))
            
            
            cv.imshow('candidate', frame)   #to show the video 
            key = cv.waitKey(1)             #to keep the window on the screen
            #print(iris_loc)
            
            if key == ord('q'):             #if we press s it will quit
                break
    #print(fpss)            
    cap.release()                           #close the camera
    cv.destroyAllWindows()                  #close all the windows which opens 

    #print(eye_closeopen) 
    #print(present_absent)
    return iris_loc, eye_closeopen, present_absent

In [10]:
# ############################################################################
# Function Name: Iris position count
# Input Parameters: list of iris location and eye blink
# Return Parameters: dataframe with all the values
# Action: It took the list inputs and saving it in a csv file       
# ############################################################################


def iris_pos_count(iris, eye): 
    with open('iris_open_close.csv','w', newline='') as file:                #opening a csv file in a write mode
        writer = csv.writer(file)                                         #writer to write a contents inside a file
        df = pd.DataFrame(list(zip(iris, eye)), columns = ['position', 'OpenAndClose'])  #zipping the values in a list
        df.to_csv(file, sep=',', encoding='utf-8-sig', index = False)     #printing the data to a csv file
        return df

In [11]:
# ############################################################################
# Function Name: Counting left right and center position and scoring
# Input Parameters: list of iris posiitons
# Return Parameters: list of count values of the iris position but in a other way
# Action: getting the word and its count and then checking if a word count occurs
#         >5 appending that to malpratice and based on len printing scoring
# ############################################################################


def count_positions(wordList):                       
    counter = 1                                        #initially its 1
    currentWord = wordList[0] if wordList else None    #check if current word is in the word list
    result = []                                        #empty list for appending a value
    malpratice = []
    
    for word in range(1, len(wordList)):      
        if currentWord == wordList[word]:              #checking if a current word matches with the list if append counter
            counter +=1
        else:
            time = counter/29.85
            result.append(f'{currentWord}:{counter}:{time}')  #printing the values to a empty list
            if currentWord == "right" and time > 0.3: #if current word is right and count value is greater than this value print
                malpratice.append("looking  at the right of the screen for long time")
            if currentWord == "left" and time > 0.3:  #if current word is left and count value is greater than this value print
                malpratice.append("looking  at the left of the screen for long time")
            if currentWord == "top" and time > 0.3:   #if current word is top and count value is greater than this value print
                malpratice.append("Looking at the top of the screen for long time")
            
            currentWord = wordList[word]
            counter = 1
    print(malpratice)
    print(len(malpratice))                                      #len of list to calculate score
    if len(malpratice) <= 5:                                    #if len is greater than this value print
        print("No Malpratice - score 100")
    elif len(malpratice) > 5 and len(malpratice) <= 20:
        print("Low possiblity of malpratice might happened - score 80")
    elif len(malpratice) > 20 and len(malpratice) <=45:
        print("Medium possibility and malpratice might happened - score - 50")
    else:
        print("malpraticed - score 0")
            
    print(result)        
    return result
            
#df.iris_positions.str.count('center')

In [12]:
# ############################################################################
# Function Name: Counting eye closing and opening and scoring
# Input Parameters: list of eye_blink
# Return Parameters: list of count values of the eye_blik but in a other way
# Action: getting the word and its count and then checking if a word count occurs
#         >10 appending that to malpratice and based on len printing scoring
# ############################################################################


def count_eyeposition(wordList):
    counter = 1                                        #initially its 1
    currentWord = wordList[0] if wordList else None    #check if current word is in the word list
    results = []                                        #empty list for appending a value
    malpratices = []
    
    for word in range(1, len(wordList)):      
        if currentWord == wordList[word]:              #checking if a current word matches with the list if append counter
            counter +=1
        else:
            time = counter/29.85
            results.append(f'{currentWord}:{counter}:{time}')  #printing the values to a empty list
            #if currentWord == "Eye Open" and counter > 10: #if current word is right and count value is greater than this value print
                #malpratices.append("Looking into camera and eyes open")
            if currentWord == "EyeClosed" and time > 0.3:  #if current word is left and count value is greater than this value print
                malpratices.append("Not looking into camera eye closed")
            
            currentWord = wordList[word]
            counter = 1
    print(malpratices)
    print(len(malpratices))                                      #len of list to calculate score
    if len(malpratices) <= 5:                                    #if len is greater than this value print
        print("No Malpratice - score 100")
    elif len(malpratices) > 5 and len(malpratices) <= 20:
        print("Low possiblity of malpratice might happened - score 80")
    elif len(malpratices) > 20 and len(malpratices) <=45:
        print("Medium possibility and malpratice might happened - score - 50")
    else:
        print("malpraticed - score 0")
            
    print(results)
    return results

In [13]:
# ############################################################################
# Function Name: Looking down
# Input Parameters: iris positions and eye blink list
# Return Parameters: list of down
# Action: getting a down if eye position is center and eye is closed        
# ############################################################################


def looking_down(position_iris, blink_eye):  
    down = []                                                                    #empty list to append values of down
    for word in range(1, len(position_iris)):                                    #getting a length of list
        if iris_positionss[word] == 'center' and blink_eye[word] == 'EyeClosed': #if position is center and eye closed mark it as down
            down.append('Down')
        else:
            down.append("Not looking into down")                                 #else not down
    return down

In [14]:
# ############################################################################
# Function Name: Counting eye down and scoring
# Input Parameters: list of eyedown
# Return Parameters: list of count values of the eye_down but in a other way
# Action: getting the word and its count and then checking if a word count occurs
#         >5 appending that to malpratice and based on len printing scoring
# ############################################################################


def count_downposition(wordList):
    counter = 1                                        #initially its 1
    currentWord = wordList[0] if wordList else None    #check if current word is in the word list
    results = []                                        #empty list for appending a value
    malpratices = []
    
    for word in range(1, len(wordList)):      
        if currentWord == wordList[word]:              #checking if a current word matches with the list if append counter
            counter +=1
        else:
            time = counter/29.85
            results.append(f'{currentWord}:{counter}:{time}')  #printing the values to a empty list
            if currentWord == "Down" and time > 0.3:  #if current word is left and count value is greater than this value print
                malpratices.append("Not looking into camera looking down")
            
            currentWord = wordList[word]
            counter = 1
    print(malpratices)
    print(len(malpratices))                                      #len of list to calculate score
    if len(malpratices) <= 5:                                    #if len is greater than this value print
        print("No Malpratice - score 100")
    elif len(malpratices) > 5 and len(malpratices) <= 20:
        print("Low possiblity of malpratice might happened - score 80")
    elif len(malpratices) > 20 and len(malpratices) <=45:
        print("Medium possibility and malpratice might happened - score - 50")
    else:
        print("malpraticed - score 0")
            
    #print(len(results))
    return results

In [15]:
video_path = ""                             #path which contains the video
video_name = "mannemlevi.mp4"               #video name

#mannemlevi.mp4               - Worked
#jawalineeraja03.mp4          - Worked
#dhanunjayaraodavuluri.mp4    - worked fine but previously got stucked
#hitesh.rp09.mp4              - worked
#neethusinghrajputh.mp4       - stuck
#SathvikManikyala_Video.MOV   - Worked
#atharvaagade12.mp4           - Worked
#dhanushree1672.mp4           - worked for the latest code previously stucked
#harshalyp23.mp4              - worked
#Honey.honey2181.mp4          - worked
#kalyanaraman3134.mp4         - worked
#kavinarunagiri5670.mp4       - worked
#mukhil0713.mp4               - worked
#kapeelpatil6.mp4             - worked
#patilgirishgp7.mp4           - worked
#mysiddiquie.mp4              - worked
#prakharksharma.mp4           - worked
#rahaelpriya.mp4              - worked
#rahul.ymcaust04.mp4          - worked
#rajeshmehta.kts.mp4          - worked
#salonishete1011.mp4          - worked
#sahilbanode45.mp4            - worked
#rakshithagowda1608.mp4       - Worked
#vaishpawar2411.mp4           - worked
#zainpatel0565.MOV            - worked
#vanidixit00.mp4              - worked
#thenmozhiraj1928.mp4         - worked
#sushmakarri1999.mp4          - worked
#suryagunal.mp4               - worked
#souravdoom50.mp4           - working fine but not in potrait mode but previously stucked
#sankarsh240620.mp4           - worked

In [16]:
#function calls

iris_positionss, eye_blink, pre_abs = iris_tracking(video_path, video_name)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
QObject::moveToThread: Current thread (0x55fe968405a0) is not the object's thread (0x55fe96bb0430).
Cannot move to target thread (0x55fe968405a0)

QObject::moveToThread: Current thread (0x55fe968405a0) is not the object's thread (0x55fe96bb0430).
Cannot move to target thread (0x55fe968405a0)

QObject::moveToThread: Current thread (0x55fe968405a0) is not the object's thread (0x55fe96bb0430).
Cannot move to target thread (0x55fe968405a0)

QObject::moveToThread: Current thread (0x55fe968405a0) is not the object's thread (0x55fe96bb0430).
Cannot move to target thread (0x55fe968405a0)

QObject::moveToThread: Current thread (0x55fe968405a0) is not the object's thread (0x55fe96bb0430).
Cannot move to target thread (0x55fe968405a0)

QObject::moveToThread: Current thread (0x55fe968405a0) is not the object's thread (0x55fe96bb0430).
Cannot move to target thread (0x55fe968405a0)

QObject::moveToThread: Current thread (0x55fe968405a0) is not 

In [17]:
positions, eyepositions = iris_pos_count(iris_positionss, eye_blink)       

In [18]:
#position_count(positions, eyepositions)    

In [19]:
eye_CLR = count_positions(iris_positionss) 

['looking  at the right of the screen for long time']
1
No Malpratice - score 100
['center:106:3.55108877721943', 'left:1:0.03350083752093802', 'center:273:9.14572864321608', 'left:1:0.03350083752093802', 'center:56:1.8760469011725291', 'left:1:0.03350083752093802', 'center:12:0.40201005025125625', 'right:1:0.03350083752093802', 'center:2:0.06700167504187604', 'right:11:0.3685092127303182', 'center:195:6.532663316582914', 'left:1:0.03350083752093802', 'center:144:4.824120603015075', 'right:3:0.10050251256281406']


In [20]:
eye_OC = count_eyeposition(eye_blink)

[]
0
No Malpratice - score 100
['EyeOpen:55:1.8425460636515911', 'EyeClosed:3:0.10050251256281406', 'EyeOpen:48:1.608040201005025', 'EyeClosed:3:0.10050251256281406', 'EyeOpen:61:2.0435510887772192', 'EyeClosed:3:0.10050251256281406', 'EyeOpen:51:1.708542713567839', 'EyeClosed:4:0.13400335008375208', 'EyeOpen:35:1.1725293132328307', 'EyeClosed:4:0.13400335008375208', 'EyeOpen:36:1.2060301507537687', 'EyeClosed:2:0.06700167504187604', 'EyeOpen:38:1.2730318257956448', 'EyeClosed:3:0.10050251256281406', 'EyeOpen:34:1.1390284757118927', 'EyeClosed:2:0.06700167504187604', 'EyeOpen:14:0.4690117252931323', 'EyeClosed:4:0.13400335008375208', 'EyeOpen:36:1.2060301507537687', 'EyeClosed:3:0.10050251256281406', 'EyeOpen:24:0.8040201005025125', 'EyeClosed:1:0.03350083752093802', 'EyeOpen:62:2.0770519262981573', 'EyeClosed:4:0.13400335008375208', 'EyeOpen:39:1.3065326633165828', 'EyeClosed:3:0.10050251256281406', 'EyeOpen:50:1.675041876046901', 'EyeClosed:4:0.13400335008375208', 'EyeOpen:33:1.10552

In [21]:
down_down = looking_down(iris_positionss, eye_blink)

In [22]:
count_downposition(down_down)

[]
0
No Malpratice - score 100


['Not looking into down:54:1.809045226130653',
 'Down:3:0.10050251256281406',
 'Not looking into down:49:1.641541038525963',
 'Down:2:0.06700167504187604',
 'Not looking into down:61:2.0435510887772192',
 'Down:3:0.10050251256281406',
 'Not looking into down:51:1.708542713567839',
 'Down:4:0.13400335008375208',
 'Not looking into down:35:1.1725293132328307',
 'Down:4:0.13400335008375208',
 'Not looking into down:36:1.2060301507537687',
 'Down:2:0.06700167504187604',
 'Not looking into down:38:1.2730318257956448',
 'Down:3:0.10050251256281406',
 'Not looking into down:35:1.1725293132328307',
 'Down:1:0.03350083752093802',
 'Not looking into down:14:0.4690117252931323',
 'Down:4:0.13400335008375208',
 'Not looking into down:36:1.2060301507537687',
 'Down:1:0.03350083752093802',
 'Not looking into down:1:0.03350083752093802',
 'Down:1:0.03350083752093802',
 'Not looking into down:87:2.914572864321608',
 'Down:4:0.13400335008375208',
 'Not looking into down:39:1.3065326633165828',
 'Down:3